In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

#Import Dataset

In [6]:
df = pd.read_csv('course_clean.csv')
df

,userId,courseId,course_name,rating
0,21461,398,pap,5
1,31909,340,machine-learning-projects,5
2,36819,588,wind-energy,4
3,10995,372,music-business-foundations,5
4,22336,209,forensic-science,5
...,...,...,...,...
49995,35239,231,getting-started-with-essay-writing,5
49996,9936,186,excel-vba-for-creative-problem-solving-part-1,5
49997,10116,439,python,5
49998,7039,513,sustainability,5


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userId       50000 non-null  int64 
 1   courseId     50000 non-null  int64 
 2   course_name  50000 non-null  object
 3   rating       50000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [8]:
df.shape

(50000, 4)

In [9]:
df.nunique()

userId         40602
courseId         595
course_name      595
rating             5
dtype: int64

#Splitting Data

In [12]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

#Pemodelan

In [13]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'courseId', 'rating']], reader)

In [14]:
"Uji 1"
benchmark = []
# Iterate over all algorithms

algorithms = [SVD(), KNNBasic()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=True)
    # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')

Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x000001980306EAD0>, <surprise.prediction_algorithms.knns.KNNBasic object at 0x000001980306F700>] 



Starting:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x000001980306EAD0>
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6553  0.6637  0.6788  0.6769  0.6777  0.6704  0.0094  
Fit time          1.17    1.19    1.12    1.22    1.34    1.21    0.07    
Test time         0.13    0.12    0.10    0.11    0.12    0.12    0.01    
Done:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x000001980306EAD0> 


Starting:  <surprise.prediction_algorithms.knns.KNNBasic object at 0x000001980306F700>
Computing the msd similarity matrix...


C:\Users\User\AppData\Local\Temp\ipykernel_27196\3357014991.py:16: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7046  0.6871  0.7040  0.6989  0.6991  0.6988  0.0063  
Fit time          296.58  177.92  138.38  182.36  192.04  197.46  52.83   
Test time         1.54    1.36    1.06    1.29    0.80    1.21    0.26    
Done:  <surprise.prediction_algorithms.knns.KNNBasic object at 0x000001980306F700> 



	DONE



C:\Users\User\AppData\Local\Temp\ipykernel_27196\3357014991.py:16: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [25]:
results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
results

,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.670449,1.207985,0.115409
KNNBasic,0.698757,197.457516,1.211315
